In [1]:
from importlib import reload
import sys

src_path =   '../src' # change as needed
sys.path.insert(0,src_path)

max_length = 128

In [2]:
import data_generator;reload(data_generator)

train_data, val_data, test_data = data_generator.GetData(max_length)

INFO:absl:Using /tmp/tfhub_modules to cache modules.
100%|██████████| 30000/30000 [00:09<00:00, 3280.67it/s]


        tag  cat  occurences
0    B-MISC    0          52
1     I-LOC    1       25078
2    I-MISC    2       10325
3     I-ORG    3       18131
4     I-PER    4       83425
5         O    5      526263
6  [nerCLS]    6       30000
7  [nerPAD]    7     2955780
8  [nerSEP]    8       30000
9    [nerX]    9      160946

                tag  cat  occurences
0  AFRICAN-AMERICAN    0      332699
1          EUROPEAN    1      330575
2         [raceCLS]    2       30000
3         [racePAD]    3     2955780
4         [raceSEP]    4       30000
5           [raceX]    5      160946

           tag  cat  occurences
0       FEMALE    0      330528
1         MALE    1      332746
2  [genderCLS]    2       30000
3  [genderPAD]    3     2955780
4  [genderSEP]    4       30000
5    [genderX]    5      160946



In [3]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
import model_utils; reload(model_utils)
    
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

model = model_utils.NER(max_length)
    
model.generate(bert_train_layers=4)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
bert_layer (BertLayer)          (None, None, 768)    108931396   input_ids[0][0]                  
                                                                 input_masks[0][0]            

In [4]:
train = False
debias = False

if train:

    model.fit(
        sess,
        train_data,
        val_data,
        epochs=8,
        batch_size=32,
        debias=debias
    ) 
    
else: 
    model_utils.initialize_vars(sess)

In [5]:
bias = model.getBiasedPValues(test_data, num_iterations=10000)

In [6]:
bias

{'race': 0.1085,
 'gender': 0.055,
 'race_test_statistic': 0.1308506342138156,
 'gender_test_statistic': 0.16482347207798972,
 'race_std': 0.10694999203201726,
 'gender_std': 0.10619998313630895,
 'race_effect_size': 0.385915253004161,
 'gender_effect_size': 0.4861106888031099}

In [7]:
sess.close()